In [9]:
import little_mallet_wrapper as lmw
import pandas as pd

%matplotlib inline

path_to_mallet = '~/mallet/bin/mallet'
path_to_proj = '../'
title_to_train = "ovid"
INPUT = "../topic-modeling/input/ovid.tsv"
output_directory_path = path_to_proj + 'topic-modeling/output'

In [10]:
train_df = pd.read_csv(INPUT, sep='\t')
training_data = [lmw.process_string(t) for t in train_df['text'].tolist()]
training_data = [d for d in training_data if d.strip()]

## LDA result analysis

In [11]:
num_topics = 20

In [12]:
# Examining the topics extracted from the data ---- 
topic_keys = lmw.load_topic_keys(output_directory_path + '/mallet.topic_keys.' + str(num_topics))

for i, t in enumerate(topic_keys):
    print(i, '\t', ' '.join(t[:10]))

0 	 tree oak branches leaves without golden bark trunk made tall
1 	 earth fire air sky sun light clouds heavens stars world
2 	 spear wound hand blood fell enemy perseus sword chest one
3 	 fate sister husband tears procne tereus alcyone ceyx philomela poor
4 	 minos crete strength flower cephalus cretan lock aeacus male purple
5 	 wings bird white birds swift changed feathers air flight caught
6 	 city son king people came god left temple sacred walls
7 	 sea waves waters land shore ocean ship wind water curved
8 	 death blood life mother shall dead body man spirit grief
9 	 hair body face like saw hands arms back still could
10 	 night sleep black thousand house rest light ears day mercury
11 	 father gods would one son said let great jupiter words
12 	 wild blood first creature woods fields boar trees jaws fierce
13 	 love girl would might said many though one marriage beauty
14 	 food twin wine mount dragons trembling sent ceres famous lying
15 	 goddess waters nymphs water river 

In [13]:
topic_distributions = lmw.load_topic_distributions(output_directory_path + '/mallet.topic_distributions.' + str(num_topics))
for p, d in lmw.get_top_docs(training_data, topic_distributions, topic_index=0, n=3):
    print(round(p, 4), d)
    print()

0.5994 gods heaven met humble household gods stooping passed low doorway old man pulled bench requested rest limbs bench baucis threw rough blanket raked warm ashes hearth brought yesterday fire life feeding leaves dried bark nursing flames aged breath pulled finely divided twigs dry stems roof breaking pushed small bronze pot next stripped leaves vegetables husband gathered well watered garden used two pronged stick lift wretched looking chine meat hanging blackened beam cutting meagre piece carefully saved chine put cut seethe boiling water

0.5648 knew well would regret flight would condemn efforts hold half mountain mine deep caves natural rock winter felt midsummer sun apples weigh branches golden purple grapes trailing vines keep pick ripe strawberries born woodland shadows autumn cherries plums juicy blue purples also large yellow ones colour fresh bees wax lack fruit wild strawberry trees tall chestnuts every tree serve

0.4972 hill hill wide area level ground turfed fresh blad

In [14]:
topic_word_probability_dict = lmw.load_topic_word_distributions(output_directory_path + '/mallet.word_weights.' + str(num_topics))
for _topic, _word_probability_dict in topic_word_probability_dict.items():
    print('Topic', _topic)
    for _word, _probability in sorted(_word_probability_dict.items(), key=lambda x: x[1], reverse=True)[:5]:
        print(round(_probability, 4), '\t', _word)
    print()

Topic 0
0.0321 	 tree
0.0141 	 oak
0.0137 	 branches
0.0116 	 leaves
0.0111 	 without

Topic 1
0.0343 	 earth
0.0208 	 fire
0.014 	 air
0.0127 	 sky
0.0104 	 sun

Topic 2
0.0224 	 spear
0.0136 	 wound
0.0125 	 hand
0.0119 	 blood
0.0111 	 fell

Topic 3
0.0119 	 fate
0.0113 	 tears
0.0113 	 husband
0.0113 	 sister
0.0095 	 procne

Topic 4
0.0151 	 minos
0.0119 	 crete
0.0095 	 strength
0.0087 	 flower
0.0079 	 cretan

Topic 5
0.04 	 wings
0.0255 	 bird
0.0125 	 white
0.012 	 changed
0.012 	 swift

Topic 6
0.0244 	 city
0.0159 	 son
0.0128 	 king
0.0123 	 people
0.0116 	 came

Topic 7
0.0499 	 sea
0.026 	 waves
0.0218 	 waters
0.0153 	 land
0.0146 	 shore

Topic 8
0.0258 	 death
0.0202 	 blood
0.0182 	 life
0.0159 	 mother
0.0115 	 shall

Topic 9
0.0112 	 hair
0.0105 	 body
0.0103 	 face
0.0097 	 like
0.0094 	 saw

Topic 10
0.0265 	 night
0.024 	 sleep
0.0102 	 black
0.0092 	 house
0.0092 	 thousand

Topic 11
0.0204 	 father
0.0178 	 gods
0.0135 	 would
0.0133 	 one
0.0105 	 son

Topic 1

In [15]:
lmw.get_js_divergence_topics(0, 7, topic_word_probability_dict)

0.7042070154810324